In [2]:
import pandas, nltk, string, numpy, csv
from pattern.fr import parse, split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import linalg
from sklearn.cluster import KMeans
import collections
from pprint import pprint

In [3]:
from langdetect import detect

In [4]:
bnp=pandas.read_csv('bnp.csv', encoding = 'utf-8').drop_duplicates()

In [5]:
bnp

,reviewHeadline,authorJobLocation,reviewDescription,company,compensationAndBenefitsRating,authorJob,cultureAndValuesRating,workLifeBalanceRating,careerOpportunitiesRating,seniorLeadershipRating,reviewDate
0,environnement de travail agréable,Paris 10,poste intéressant avec beaucoup de formations....,BNP Paribas,0,Gestionnaire Crédits Structurés (Ancien employ...,0,0,0,0,6 juillet 2015
1,Agréable quand on va dans le sens de la hiérar...,Champigny-sur-Marne (94),il est agréable de travailler chez BNP Paribas...,BNP Paribas,4,"Conseiller clientèle (Ancien employé),",2,4,4,2,25 novembre 2015
2,Bon environnement de travail,"Paris, Montreuil","J'y travaillais beaucoup, mais avec des person...",BNP Paribas,0,"Consultant (Ancien employé),",0,0,0,0,24 novembre 2015
3,AMOA GED dematérialisation,"Paris, A8","Equipe sympa.,Ambiance correcte,Envoi de flux...",BNP Paribas,4,Consultant AMOA GED-Dématérialisation (Employé...,1,4,1,1,21 novembre 2015
4,stage agréable,"Rueil-Malmaison, A8","Stage d'été agréable, des premières missions a...",BNP Paribas,0,"Auxiliaire d'été (Ancien employé),",0,0,0,0,3 novembre 2015
5,emploi,val d oise et oise,"periode trop courte pour en juger.,Mais bonne ...",BNP Paribas,2,"accueil clientele (Ancien employé),",2,2,2,2,24 octobre 2015
6,environnement de travail agréable,joinville le pont,"à l'agence de joinville le pont, l'environneme...",BNP Paribas,3,"Gestion de l'accueil (Ancien employé),",3,4,2,3,16 octobre 2015
7,Expérience professionnel très agréable,Montreuil,Ma journée consistait à effectuer une surveill...,BNP Paribas,4,"Technicien D'exploitation (Ancien employé),",5,4,4,5,7 octobre 2015
8,Bonne ambiance générale,Paris,"Au sein de la BNP Paribas, j'ai eu la chance d...",BNP Paribas,5,"Gestionnaire Administratif (Ancien employé),",4,4,4,4,6 octobre 2015
9,cetelem,marseille,"Bonne entreprise,Très bonne ambiance au sein d...",BNP Paribas,0,"Auxiliaire de vacances (Ancien employé),",0,0,0,0,5 octobre 2015


In [6]:
bnp1=bnp[['reviewDescription','authorJobLocation','authorJob','reviewDate']]

In [7]:
bnp1

,reviewDescription,authorJobLocation,authorJob,reviewDate
0,poste intéressant avec beaucoup de formations....,Paris 10,Gestionnaire Crédits Structurés (Ancien employ...,6 juillet 2015
1,il est agréable de travailler chez BNP Paribas...,Champigny-sur-Marne (94),"Conseiller clientèle (Ancien employé),",25 novembre 2015
2,"J'y travaillais beaucoup, mais avec des person...","Paris, Montreuil","Consultant (Ancien employé),",24 novembre 2015
3,"Equipe sympa.,Ambiance correcte,Envoi de flux...","Paris, A8",Consultant AMOA GED-Dématérialisation (Employé...,21 novembre 2015
4,"Stage d'été agréable, des premières missions a...","Rueil-Malmaison, A8","Auxiliaire d'été (Ancien employé),",3 novembre 2015
5,"periode trop courte pour en juger.,Mais bonne ...",val d oise et oise,"accueil clientele (Ancien employé),",24 octobre 2015
6,"à l'agence de joinville le pont, l'environneme...",joinville le pont,"Gestion de l'accueil (Ancien employé),",16 octobre 2015
7,Ma journée consistait à effectuer une surveill...,Montreuil,"Technicien D'exploitation (Ancien employé),",7 octobre 2015
8,"Au sein de la BNP Paribas, j'ai eu la chance d...",Paris,"Gestionnaire Administratif (Ancien employé),",6 octobre 2015
9,"Bonne entreprise,Très bonne ambiance au sein d...",marseille,"Auxiliaire de vacances (Ancien employé),",5 octobre 2015


In [8]:
bnp1['language'] = (bnp1['reviewDescription']).apply(lambda x: detect(x))

/anaconda/envs/Python2.7/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
bnp1['OriginalReview'] = bnp1['reviewDescription']

/anaconda/envs/Python2.7/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [10]:
bnp_fr=bnp1[bnp1['language']=='fr']

In [116]:
bnp_fr.loc[:,'review_lower'] = bnp_fr['reviewDescription'].apply(lambda x: x.lower()).str.replace(',', ' ')

//anaconda/envs/Python2.7/lib/python2.7/site-packages/pandas/core/indexing.py:508: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [119]:
bnp_fr['review_tokens'] = bnp_fr['review_lower'].apply(lambda x: parse(x.replace('.',""), chunks = False, lemmata = True).split()[0])

/anaconda/envs/Python2.7/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [120]:
bnp_fr['review_tokens']

0      [[poste, NN, poste], [intéressant, VBG, intére...
1      [[il, PRP, il], [est, VB, être], [agréable, JJ...
2      [[j', PRP, je], [y, PRP, y], [travaillais, VB,...
3      [[equipe, NN, equipe], [sympa, JJ, sympa], [am...
4      [[stage, NN, stage], [d', IN, de], [été, NN, é...
5      [[periode, NN, periode], [trop, RB, trop], [co...
6      [[à, IN, à], [l', DT, le], [agence, NN, agence...
7      [[ma, PRP$, mum], [journée, NN, journée], [con...
8      [[au, IN, au], [sein, NN, sein], [de, IN, de],...
9      [[bonne, JJ, bon], [entreprise, NN, entreprise...
10     [[le, DT, le], [poste, NN, poste], [que, IN, q...
11     [[journée, NN, journée], [de, IN, de], [travai...
12     [[développement, NN, développement], [et, CC, ...
13     [[-, -, -], [rythme, NN, rythme], [de, IN, de]...
14     [[excellente, JJ, excellent], [relation, NN, r...
15     [[préparation, NN, préparation], [de, IN, de],...
16     [[je, PRP, je], [suis, VB, suivre], [très, RB,...
18     [[les, DT, le], [collègu

In [121]:
bnp_fr['review_lemmas'] =\
bnp_fr['review_tokens'].apply(lambda x: [t[2] for t in x if t[1] not in ['DT',\
'CC','IV','IN','PRP','PRP$','SYM','UH',',','.',':','(',')','-'] ])

/anaconda/envs/Python2.7/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [122]:
bnp_fr['review_lemmas']=bnp_fr['review_lemmas'].apply(lambda x: [t for t in x if t not in stopwords.words('french')])

/anaconda/envs/Python2.7/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [123]:
bnp_fr['review'] = bnp_fr['review_lemmas'].apply(lambda x: " ".join(x))

/anaconda/envs/Python2.7/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [124]:
bnp_fr['review']

0      poste intéresser beaucoup formation bon entent...
1      être agréable travailler bnp pariba projet per...
2                   travailler beaucoup person compétent
3      equipe sympa ambiance correcte envoi flux xml ...
4      stage agréable premier mission adaptéer niveau...
5      periode trop courte juger bon experience bon r...
6      agence joinville pont environnement travail êt...
7      journée consister effectuer surveillance compt...
8      sein bnp pariba avoir avoir chance être tutell...
9       bon entreprise très bon ambiance sein entreprise
10     poste occuper actuellement être transversal tâ...
11     journée travail agréable globalité personnel s...
12     développement formation être opportunité évolu...
13     rythme travail soutenir parfait développement ...
14     excellent relation différent service différent...
15     préparation journée préparer poste travail occ...
16     suivre très satisfaire journée travail bnp pariba
18     collègue être gentil tra

In [108]:
bnp_fr['review_lemmas']

0      [poste, intéresser, beaucoup, formation, bon, ...
1      [être, agréable, travailler, bnp, pariba, proj...
2              [travailler, beaucoup, person, compétent]
3      [equipe, sympa,ambiance, correcte,envoi, flux,...
4      [stage, agréable, premier, mission, adaptéer, ...
5      [periode, trop, courte, juger,mai, bon, experi...
6      [agence, joinville, pont, environnement, trava...
7      [journée, consister, effectuer, surveillance, ...
8      [sein, bnp, pariba, avoir, avoir, chance, être...
9      [bon, entreprise,trè, bon, ambiance, sein, ent...
10     [poste, occuper, actuellement, être, transvers...
11     [journée, travail, agréable, globalité,personn...
12     [développement, formation, être, opportunité, ...
13     [rythme, travail, soutenir, parfait, développe...
14     [excellent, relation, différent, service, diff...
15     [préparation, journée, préparer, poste, travai...
16     [suivre, très, satisfaire, journée, travail, b...
18     [collègue, être, gentils

# Création du Bag of Words

La suppression des mots comme 'je' à l'aide du POS n'est pas efficace. Le mieux est d'établire une liste des mots vides à supprimer. Pour cela on peut utiliser la liste de stop words de nltk

In [19]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(bnp_fr['review'])

In [21]:
vocab = vectorizer.get_feature_names()
nbMots = numpy.sum(X.toarray(),axis=0)

vocab[-200:]

[u'souci',
 u'soucieux',
 u'souder',
 u'souhaiter',
 u'souple',
 u'source',
 u'sourire',
 u'sous',
 u'soutenir',
 u'souvenir',
 u'souvent',
 u'spinoff',
 u'sp\xe9cialis\xe9er',
 u'sp\xe9cifique',
 u'st',
 u'stable',
 u'stage',
 u'stagiaire',
 u'statistique',
 u'statut',
 u'strat\xe9gie',
 u'strict',
 u'structure',
 u'structur\xe9',
 u'succursal',
 u'succursale',
 u'suffisamment',
 u'suite',
 u'suivi',
 u'suivre',
 u'sujet',
 u'superbe',
 u'suppl\xe9mentaire',
 u'sup\xe9rieur',
 u'sur',
 u'surcharge',
 u'surtout',
 u'surveillance',
 u'symbiose',
 u'sympa',
 u'sympathie',
 u'sympathique',
 u'sympathiques',
 u'synonyme',
 u'synth\xe8se',
 u'syst\xe8me',
 u's\xe9curisation',
 u's\xe9curit\xe9',
 u's\xe9rieux',
 u'tableau',
 u'taches',
 u'taille',
 u'talent',
 u'talents',
 u'talentueux',
 u'tant',
 u'tarifaire',
 u'tarificatrice',
 u'technique',
 u'techniques',
 u'temporaire',
 u'temps',
 u'terme',
 u'terminer',
 u'texte',
 u'ticketing',
 u'tickets',
 u'tier',
 u'titre',
 u'titres',
 u'tomb

In [22]:
import operator

In [23]:
bnp_fr.iloc[130]['reviewDescription']

u'Une entreprise leader sur le march\xe9 mais qui n\'embauche pas les "bons" profils. En effet, l\'entreprise ne fait pas de r\xe9tention des bons talents et ne propose pas un des opportunit\xe9s d\'\xe9volutions comme cela est dit \xe0 l\'embauche.,Par contre, bon \xe9quilibre entre vie priv\xe9e et vie professionnelle.'

In [24]:
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(X)

In [25]:
nbMots2 = numpy.sum(tfidf.toarray(),axis=0)

dictionary2 = dict(zip(vocab, nbMots2))
print sorted(dictionary2.items(), key=operator.itemgetter(1))[-20:]

[(u'management', 3.527490273911146), (u'pariba', 3.5393633753193474), (u'journ\xe9e', 3.6162558990728408), (u'environnement', 3.674203986632854), (u'sein', 3.8363854502134775), (u'aspect', 4.0300528977342154), (u'exp\xe9rience', 4.1146444154642268), (u'bnp', 4.2268511854043496), (u'ambiance', 4.4114471794710672), (u'poste', 4.7519625270714982), (u'collaborateur', 4.9169983616573294), (u'plus', 5.0536626362703005), (u'\xe9quipe', 5.5542396713978146), (u'entreprise', 5.9447665555378233), (u'agr\xe9able', 6.5133360896430332), (u'avoir', 7.9502284756701469), (u'tr\xe8s', 8.655740104087533), (u'travail', 9.1711368140836989), (u'bon', 9.4537062393561673), (u'\xeatre', 14.348710158572455)]


# Décomposition SVD

In [125]:
U, s, Vt = linalg.svds(tfidf, k = 74)

In [27]:
Vt.shape

(74, 1328)

In [28]:
len([w for w in s if w > s.mean()])

26

In [29]:
sd = numpy.zeros((74,74))

In [30]:
numpy.fill_diagonal(sd, s)

In [31]:
sd

array([[ 0.95981613,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.96325461,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.96626265, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  1.74361698,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.95999057,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  3.08606425]])

In [32]:
VS = numpy.dot(numpy.transpose(Vt),sd)

In [33]:
VS.shape

(1328, 74)

### Clustering K-means

In [34]:
km_model = KMeans(n_clusters=35)

In [35]:
km_model.fit(VS)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=35, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [36]:
clustering = collections.defaultdict(list)

In [37]:
for idx, label in enumerate(km_model.labels_):
    clustering[label].append(idx)

In [38]:
d ={}

In [39]:
for label in clustering :
    d[label] = []
    for i in clustering[label]:
        d[label] = d[label]+[vectorizer.get_feature_names()[i]]

In [40]:
writer = csv.writer(open('dict.csv','wb'))
for key, value in d.items():
    writer.writerow([key, value])

### Printing du résultat

In [41]:
for k in range(0,len(d)):
    print '---------'+'cluster '+ str(k)
    for i in  clustering[k]:
        print vectorizer.get_feature_names()[i]

---------cluster 0
analyse
bilan
clientèle
crédit
disponibilité
dossier
former
management
pme
préparation
---------cluster 1
accueillant
actuariat
adaptéer
affaire
ageable
agence
ainsi
amenére
améliorer
année
antin
apporteur
apprentissage
archivage
arriver
assurance
attitrer
attractif
aussi
bancaire
bancaires
brève
busines
but
calculon
campagne
cardif
cependant
chance
changéer
chose
classement
classer
clientele
coaching
collectivement
collègues
commencer
concerner
connaitre
conseiller
considéréer
convivial
coter
courant
court
courtoises
devoir
direct
direction
débattre
déjà
démarchage
désaccord
détection
effort
entier
envie
espèce
et
etant
exigeant
facilement
fermeture
feutrer
finalement
financement
financiers
flexible
fmi
fois
formére
fréquent
gamme
grosse
guichet
gérer
habitation
haut
image
incivilité
inconvénient
individuellement
intégrer
intérimaire
inventaire
je
jusque
long
lors
mailing
mails
manager
mathématique
matin
matinée
mieux
milieu
moins
moment
montage
moyen
niveau
notaire

In [42]:
for w in stopwords.words('french'):
    print w

au
aux
avec
ce
ces
dans
de
des
du
elle
en
et
eux
il
je
la
le
leur
lui
ma
mais
me
même
mes
moi
mon
ne
nos
notre
nous
on
ou
par
pas
pour
qu
que
qui
sa
se
ses
son
sur
ta
te
tes
toi
ton
tu
un
une
vos
votre
vous
c
d
j
l
à
m
n
s
t
y
été
étée
étées
étés
étant
étante
étants
étantes
suis
es
est
sommes
êtes
sont
serai
seras
sera
serons
serez
seront
serais
serait
serions
seriez
seraient
étais
était
étions
étiez
étaient
fus
fut
fûmes
fûtes
furent
sois
soit
soyons
soyez
soient
fusse
fusses
fût
fussions
fussiez
fussent
ayant
ayante
ayantes
ayants
eu
eue
eues
eus
ai
as
avons
avez
ont
aurai
auras
aura
aurons
aurez
auront
aurais
aurait
aurions
auriez
auraient
avais
avait
avions
aviez
avaient
eut
eûmes
eûtes
eurent
aie
aies
ait
ayons
ayez
aient
eusse
eusses
eût
eussions
eussiez
eussent


In [43]:
len(stopwords.words('french'))

155

### LSA

In [126]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

lsa = TruncatedSVD(2, algorithm = 'arpack')
dtm_lsa = lsa.fit_transform(tfidf)
dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)
lsa.explained_variance_ratio_.sum()

0.03444249778620713

In [127]:
vec = pandas.DataFrame(lsa.components_,index=["component_1","component_2" ],columns = vectorizer.get_feature_names())
vec

,06h45,100,1000,12h,13h,14h,17an,17h,17h30,18h30,...,état,étendre,étude,étudier,éventuel,évidence,évoluer,évoluon,évolution,être
component_1,0.007604,0.004945,0.004332,0.010328,0.015875,0.007699,0.009590,0.019835,0.004398,0.007604,...,0.018093,0.004101,0.012869,0.011920,0.001343,0.002639,0.056008,0.003597,0.071680,0.438150
component_2,-0.008537,0.001838,-0.008827,-0.005443,-0.011676,-0.009341,-0.020433,-0.001022,-0.002742,-0.008537,...,0.017913,-0.004496,-0.007059,-0.009301,-0.003177,-0.002758,-0.030521,-0.005831,0.035012,-0.292565


In [128]:
pandas.DataFrame(dtm_lsa, index = bnp_fr['review'], columns = ["component_1","component_2" ])

,component_1,component_2
review,,
poste intéresser beaucoup formation bon entente collégue surcharge travail,0.758746,0.651386
être agréable travailler bnp pariba projet personnel prendre dessus professionnel convier forcément hiérarchie plans carrière,0.953274,-0.302108
travailler beaucoup person compétent,0.993126,-0.117047
equipe sympa ambiance correcte envoi flux xml reconnaissance documentaire gestion flux workflox rad lad workflow,0.897852,0.440297
stage agréable premier mission adaptéer niveau étude,0.856778,0.515686
periode trop courte juger bon experience bon retour,0.430480,0.902600
agence joinville pont environnement travail être très agréable équipe jeun souder très bon état esprit,0.712329,0.701846
journée consister effectuer surveillance compte rendre toutes alerte généréer différent application gérer société bnp pariba ainsi traiter tous incident niveau 1 escalader autres différent pôle charge management toujours être très écouter apprendre beaucoup avoir toujours avoir chance travailler bon collaborateur toujours disponible prêt aider équipe aspect plus difficile poste être pression génère aspect plus agréable être sentir utile pouvoir apprendre toujours nouvel choses,0.966572,-0.256396
sein bnp pariba avoir avoir chance être tutelle collaborateur talentueux vrai envie transmettre savoir-faire ainsi suivre facilement intégrer société avoir pouvoir très vite commencer effectuer tâche passionnant mailing intern collaborateur extern ainsi gestion colis poste avoir occuper demander grand rigueur sen organisation permettre acquérir grand autonomie,0.829606,-0.558349


### LDA

from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = 2000
n_features = 1000
n_topics = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()


# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.


# Fit the NMF model
print("Fitting the NMF model with tf-idf features,"
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)
exit()
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model:")
tfidf_feature_names = vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)


### NLTK

In [131]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
#trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = BigramCollocationFinder.from_words(txt)
finder.nbest(bigram_measures.pmi, 20)# doctest: +NORMALIZE_WHITESPACE

[(u'18h30', u'19h00'),
 (u'360', u'degr\xe9'),
 (u'800', u'1000'),
 (u'aboutir', u'promotion'),
 (u'accueillant', u'n\xe9cessiter'),
 (u'acteur', u'incontournable'),
 (u'aff\xe9rent', u'dernier'),
 (u'agricole', u'distinguer'),
 (u'alerte', u'g\xe9n\xe9r\xe9er'),
 (u'al\xe9atoire', u'cinq'),
 (u'ambitieux', u'outre'),
 (u'am\xe9lioration', u'ma\xeetrise'),
 (u'analyser', u'repartir'),
 (u'appliqu\xe9er', u'convenablement'),
 (u'apporteur', u'affaire'),
 (u'aprimo', u'arm'),
 (u'argentin', u'sanction'),
 (u'attitude', u'certes'),
 (u'augmenter', u'statut'),
 (u'auxiliaire', u'vacance')]

In [161]:
import itertools
blocklist =['u\xeatre','avoir','tres','plus','bon']
corp = bnp_fr['review_lemmas'].map(lambda x : [t for t in x if t not in blocklist])
corp = list(itertools.chain.from_iterable(corp.tolist()))

txt = nltk.Text(corp)
txt.collocations()

bnp pariba; aspect difficile; aspect agréable; opportunité
développement; journée travail; ambiance travail; difficile poste;
miser place; environnement travail; culture entreprise; possibilité
évolution; très sympathique; fois semaine; pression commercial; poste
occuper; 12h 13h; exécution ordre; permettre acquérir; cohésion
équipe; compte rendre


/anaconda/envs/Python2.7/lib/python2.7/site-packages/ipykernel/__main__.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  app.launch_new_instance()


In [166]:
txt

<Text: poste int\xe9resser beaucoup formation entente coll\xe9gue surcharge travail...>

# Gensim

In [46]:
import gensim

In [153]:
#blocklist = ['\xeatre','avoir','tr\xe8s']
#bnp_fr['review2'] = bnp_fr['review'].apply(lambda x: [t for t in x if t not in blocklist])
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corp)

corpus=vectorizer.get_feature_names()
id2words = dict((v, k) for k, v in vectorizer.vocabulary_.iteritems())
corpus_gensim = gensim.matutils.Sparse2Corpus(X, documents_columns=False)


In [154]:
lda = gensim.models.ldamodel.LdaModel(corpus_gensim, id2word=id2words, num_topics=5, update_every=1, chunksize=1000, passes=3)
lda.print_topics()

[(0,
  u'0.049*collaborateur + 0.032*bnp + 0.023*formation + 0.020*commercial + 0.019*sein + 0.017*beaucoup + 0.015*tous + 0.014*d\xe9veloppement + 0.011*varier + 0.011*mettre'),
 (1,
  u'0.090*travail + 0.037*pouvoir + 0.034*poste + 0.028*aspect + 0.024*coll\xe8gue + 0.021*agence + 0.019*temps + 0.018*opportunit\xe9 + 0.018*t\xe2che + 0.017*employ\xe9'),
 (2,
  u'0.070*tr\xe8s + 0.027*faire + 0.025*environnement + 0.023*\xe9volution + 0.022*journ\xe9e + 0.021*ambiance + 0.019*service + 0.018*exp\xe9rience + 0.011*compte + 0.010*donner'),
 (3,
  u'0.215*\xeatre + 0.033*clients + 0.027*management + 0.027*gestion + 0.025*pariba + 0.022*groupe + 0.021*permettre + 0.015*toujours + 0.013*aussi + 0.013*banque'),
 (4,
  u'0.051*agr\xe9able + 0.036*\xe9quipe + 0.035*entreprise + 0.023*travailler + 0.021*professionnel + 0.018*possibilit\xe9 + 0.016*contact + 0.016*autre + 0.015*difficile + 0.015*diff\xe9rent')]

### Pattern

In [54]:
from pattern.metrics import cooccurrence

In [148]:
m = cooccurrence(corp, window=(-3, -3),
      term1 = lambda w: True,
      term2 = lambda w: True,
  normalize = lambda w: w)

In [151]:
m

{u'succursale': {u'3': 1},
 u'reprendre': {u'fin': 1},
 u'code': {u'faire': 1},
 u'participation': {u'etre': 1},
 u'maximum': {u'difficile': 1},
 u'dessus': {u'projet': 1, u'venir': 1},
 u'global': {u'fin': 1},
 u'ost': {u'dvse': 1},
 u'\u20ac': {u'cr\xe9dit': 1},
 u'baisse': {u'rdv': 1},
 u'int\xe9rimaire': {u'beaucoup': 1},
 u'retour': {u'courte': 1, u'parole': 1},
 u'\xe9voluon': {u'poste': 1},
 u'aider': {u'd\xe9butant': 1, u'toujours': 1, u'tr\xe8s': 1},
 u'limiter': {u'hi\xe9rarchiser': 1, u'responsables': 1},
 u'gm': {u'travailler': 1},
 u'faute': {u'dessus': 1},
 u's\xe9curit\xe9': {u'pee': 1},
 u'entendre': {u'groupe': 1},
 u'sympa': {u'person': 1},
 u'obliger': {u'trop': 1},
 u'facilement': {u'savoir-faire': 1},
 u'fran\xe7ais': {u'employeur': 1, u'r\xe9daction': 1},
 u'devenir': {u'aspect': 1, u'opportunit\xe9': 1},
 u'pouvoir': {u'bnp': 1,
  u'candidat': 1,
  u'carri\xe8re': 1,
  u'cependant': 1,
  u'circonstance': 1,
  u'connaissance': 1,
  u'convenablement': 1,
  u'dialog

# Text summarization

In [291]:
from gensim.summarization import summarize, keywords
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [297]:
#corp = bnp_fr['reviewDescription'].map(lambda x : [t for t in x if t not in blocklist])
# -*- coding: utf-8 -*-
import unicodedata

corp = bnp_fr['reviewDescription'].str.cat(sep='. ')
titre = bnp['reviewHeadline'].str.cat(sep='. ')
bnp['total'] = bnp['reviewHeadline'].str.cat(bnp['reviewDescription'].values.astype(str), sep='. ')
total = bnp['total'].str.cat(sep='. ')

corp2 = unicodedata.normalize('NFKD', total).encode('ascii','ignore')

In [300]:
corp2
#summarize(corp2, ratio=0.01)
#print keywords(corp, ratio=0.01)

'environnement de travail agreable. poste interessant avec beaucoup de formations. bonne entente avec les collegues malgre une surcharge de travail.. Agreable quand on va dans le sens de la hierarchie. il est agreable de travailler chez BNP Paribas mais des que notre projet personnel prend le dessus sur le professionnel, cela ne convient pas forcement a la hierarchie et ses plans de carriere.. Bon environnement de travail. J\'y travaillais beaucoup, mais avec des personnes competentes.. AMOA GED dematerialisation. Equipe  sympa.,Ambiance correcte,Envoi de flux xml pour la reconnaissance documentaire et gestion des flux workflox.,RAD, LAD et workflow. stage agreable. Stage d\'ete agreable, des premieres missions adaptees au niveau d\'etude.. emploi. periode trop courte pour en juger.,Mais bonne experience,Bon retour. environnement de travail agreable. a l\'agence de joinville le pont, l\'environnement de travail etait tres agreable, une equipe jeune et soude avec un tres bonne etat d\'e

# Sumy


ImportError: No module named utils